# Red Neuronal Convolucional

Vamos a introducir un tipo popular de modelo llamado una [red neuronal convolucional](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53) que es especialmente bueno para la lectura de imágenes y su clasificación.

## Objetivos   

* Preparar datos específicamente para una CNN
* Crear un modelo CNN más sofisticado, comprendiendo una mayor variedad de capas del modelo
* Entrenar un modelo CNN y observar su rendimiento

## Carga y preprocesamiento de datos

La siguiente celda contiene las técnicas de preprocesamiento de datos que se ha visto en otros notebooks. Se revisará y ejecutará antes de avanzar:

In [3]:
import tensorflow.keras as keras
import pandas as pd

# Load in our data from CSV files
train_df = pd.read_csv("datasets/asl_data/sign_mnist_train.csv")
valid_df = pd.read_csv("datasets/asl_data/sign_mnist_valid.csv")

# Separate out our target values
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# Separate out our image vectors
x_train = train_df.values
x_valid = valid_df.values

# Turn our scalar targets into binary categories
num_classes = 24
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# Normalize our image data
x_train = x_train / 255
x_valid = x_valid / 255

## Cambiando de tamaño ("Reshaping") las imágenes para la CNN

Las imágenes individuales del conjunto de datos tienen el formato de listas largas de 784 píxeles:

In [4]:
x_train.shape, x_valid.shape


((27455, 784), (7172, 784))

En este formato, no se dispone de toda la información sobre qué píxeles están cerca unos de otros. Por ello, no se puede aplicar convoluciones que detecten características. Se debe modificar el conjunto de datos para que tengan un formato de 28x28 píxeles. Esto permitirá a las convoluciones de la red asociar grupos de píxeles y detectar características importantes.

Se debe tener en cuenta que para la primera capa convolucional del modelo, hay que tener no sólo la altura y la anchura de la imagen, sino también el número de [canales de color](https://www.photoshopessentials.com/essentials/rgb/). Las imágenes son en escala de grises, por lo que sólo hay 1 canal.

Eso significa que se debe convertir la forma actual `(27455, 784)` a `(27455, 28, 28, 1)`. Por conveniencia, es posible pasar al método [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html#numpy.reshape) un `-1` para cualquier dimensión que se desee que permanezca igual, por lo tanto:

In [5]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)

In [6]:
x_train.shape

(27455, 28, 28, 1)

In [7]:
x_valid.shape

(7172, 28, 28, 1)

## Construcción de un modelo convolucional

Hoy en día, muchos científicos de datos comienzan sus proyectos tomando prestadas las propiedades de un modelo de un proyecto similar. Asumiendo que el problema no es totalmente único, hay una gran posibilidad de que la gente haya creado modelos que funcionarán bien y que están publicados en repositorios online como [TensorFlow Hub](https://www.tensorflow.org/hub) y el [NGC Catalog](https://ngc.nvidia.com/catalog/models). En este caso, se utiliza un modelo que funcionará bien para este problema.

<img src="./img/cnn.png" width=180 />

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)

model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu", 
                 input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(units=num_classes, activation="softmax"))

/usr/local/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-03-07 11:01:11.544130: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


### [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D)

<img src="img/conv2d.png" width=300 />   


Estas son las capas convolucionales 2D. Los núcleos pequeños repasan la imagen de entrada y detectan las características importantes para la clasificación. Las primeras convoluciones del modelo detectan características sencillas, como líneas. Las convoluciones posteriores detectan características más complejas. Veamos nuestra primera capa Conv2D:
```Python
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same'...)
```
El 75 se refiere al número de filtros que se entrenarán. (3,3) se refiere al tamaño de esos filtros. *Strides* se refiere al tamaño de paso (o de salto) que usará el filtro al pasar sobre la imagen. El *Padding* se refiere a si la imagen de salida que se crea a partir del filtro coincidirá con el tamaño de la imagen de entrada. 

### [Batch Normalization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization)   

Al igual que la normalización de las entradas, la normalización por lotes (*Batch Normalization*) escala los valores en las capas ocultas para mejorar el entrenamiento. [Más información aquí](https://blog.paperspace.com/busting-the-myths-about-batch-normalization/). 

### [MaxPool2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D)    

<img src="img/maxpool2d.png" width=300 />   

Max pooling takes an image and essentially shrinks it to a lower resolution. It does this to help the model be robust to translation (objects moving side to side), and also makes our model faster.

### [Dropout](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout)   

<img src="img/dropout.png" width=360 />   

**Dropout** es una técnica para evitar el sobreajuste. **Dropout** selecciona aleatoriamente un subconjunto de neuronas y las desactiva, de modo que no participan en la propagación hacia delante o hacia atrás en ese paso concreto. Esto ayuda a garantizar que la red sea robusta y redundante, y no dependa de una sola área para obtener respuestas. 

### [Flatten (Aplanamiento)](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten)   

La capa de tipo *Flatten* toma la salida de una capa, que es multidimensional, y la aplana en una matriz unidimensional. La salida se denomina vector de características y se conectará a la capa de clasificación final.

### [Dense/Full Connected](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)    

Ya se ha visto anteriormente las capas densas en modelos anteriores. La primera capa densa (512 unidades) toma el vector de características como entrada y aprende qué características contribuirán a una clasificación concreta. La segunda capa densa (24 unidades) es la capa de clasificación final que genera la predicción.

## Summarizing the Model   
Aquí se resume el modelo que se acaba de crear. Se puede observar los parámetros entrenables que el modelo tiene:

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 75)     │           750 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 75)     │           300 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 75)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 50)     │        33,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 50)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 50)     │           200 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 50)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 25)       │        11,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 7, 7, 25)       │           100 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 25)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       205,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24)             │        12,312 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 264,049 (1.01 MB)

 Trainable params: 263,749 (1.01 MB)

 Non-trainable params: 300 (1.17 KB)

## Compilación del modelo

In [ ]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"])

## Entrenamiento   

Se prepara el modelo para entrenar durante 20 épocas y comprobar la precisión del mismo:

In [11]:
model.fit(x_train, y_train, epochs=10, verbose=1, validation_data=(x_valid, y_valid))

Epoch 1/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 65s 73ms/step - accuracy: 0.7595 - loss: 0.8205 - val_accuracy: 0.9129 - val_loss: 0.2722
Epoch 2/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 56s 65ms/step - accuracy: 0.9922 - loss: 0.0256 - val_accuracy: 0.9288 - val_loss: 0.2606
Epoch 3/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 60s 70ms/step - accuracy: 0.9973 - loss: 0.0084 - val_accuracy: 0.9586 - val_loss: 0.1626
Epoch 4/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 60s 70ms/step - accuracy: 0.9982 - loss: 0.0054 - val_accuracy: 0.9380 - val_loss: 0.2465
Epoch 5/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 63s 73ms/step - accuracy: 0.9982 - loss: 0.0059 - val_accuracy: 0.9216 - val_loss: 0.3470
Epoch 6/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 57s 66ms/step - accuracy: 0.9993 - loss: 0.0025 - val_accuracy: 0.9614 - val_loss: 0.1521
Epoch 7/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 56s 65ms/step - accuracy: 0.9993 - loss: 0.0022 - val_accuracy: 0.9614 - val_loss: 0.1820
Epoch 8/10
858/858 ━━━━━━━━━━━━━━━━━━━━ 56s 65ms/step - accuracy: 0.9998 - loss: 9.8705e-0

## Resultados   

Parece que este modelo tiene una precisión de entrenamiento muy alta, y la precisión de validación también ofrece muy buenos valores.
Se puede observar que la precisión de validación da saltos. Esto indica que este modelo aún no generaliza a la perfección. 